In [13]:
!pip install langchain langchain_community langchainhub chromadb langchain-google-genai

In [14]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [15]:
%pip install -qU langchain_community beautifulsoup4

In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path=["https://www.educosys.com/course/genai"])

docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Hands-on Generative AI Course', 'language': 'en'}, page_content="Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 Weeks CourseAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Networks (GANs) Variational Autoencoders (

In [17]:
%pip install -qU langchain-text-splitters

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [19]:
print(splits[0])
print(splits[1])

page_content='Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 Weeks CourseAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mini Project - Train an Autoencoder on the MNIST Dataset2Week 2Deep Generative Models Discriminative and Generative models Generative Adversarial Networks (GANs) Variational Autoencoders (VAEs) Probabilistic Data Generation Using VAEs Four Mini Projects using TensorFlow Metrics Visualization using TensorBoard Mini Project - Implement a GAN to Generate Handwritten Di

In [20]:
print(len(splits))

11


In [21]:
%pip install chromadb

In [22]:
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [23]:
print(vectorstore._collection.get())

{'ids': ['353b4939-d6a7-48e9-9c5f-7825c1ab1389', '1899d654-b1db-4ecb-b688-2441ad59cec4', '6f904489-c22e-435e-8edc-ac4d423e7de0', '001f3086-33f8-4028-83e9-d10bbc1362b1', '330848db-e7a0-4e93-9793-8ff31ab87508', '583b46b2-75b9-4711-ba26-d4b677e11816', '539f040d-7b13-43e8-a9e4-4919221df810', '1421bbd3-0aaa-40a9-88d8-6e571cbec089', 'a0e7a9ba-968c-4f97-aaf7-1ef1ffb5cbda', 'e7388073-ffaa-4b0e-b782-f80219012240', 'e4dbea21-f75a-4a1f-91bf-191b1f90f8d9', '4ff8d076-edd8-496d-bcd3-2fd49e120588', '0477182c-e1ec-413e-a1d5-21fa97b0b19a', '1cf9a83d-a5dc-402e-b0b2-c8eec8178776', 'f6b8aa31-492f-4cf3-b9a0-5b3315f62476', 'e5993a86-a1a6-4cb5-b4a2-bb3a4b773c8e', '43f0c51f-319d-46cd-8d47-aec7b4cc8dcc', 'f1259841-8906-4588-80a2-70bb15f70a51', '6ebf5c30-36ab-407d-8f81-aa78825125cf', '66d29929-d9e9-4514-97cd-bc3a15605bc7', 'c85cd408-33b1-40fb-9f60-8c880ac53686', '2afde5b3-fee0-4d69-b37a-1895174c3f6b'], 'embeddings': None, 'documents': ['Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimo

In [24]:
retriever = vectorstore.as_retriever()

In [25]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
)

In [28]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [29]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [31]:
rag_chain = ({"context":retriever | format_docs, "question":RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [35]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

'Yes, the course recordings are available. The context does not specify how long the recordings are available. You can also go through notes offline.'

In [36]:
rag_chain.invoke("Testimonials available ? ")

'Yes, testimonials are available from Syed I, Ruthira Sekar, Abhijit Mone, and Manika Kaushik. These testimonials praise the course and the instructor, Keerti.'

In [38]:
from langchain_core.runnables import RunnableLambda

In [40]:
def print_prompt(prompt_text):
  print("Prompt - ", prompt_text)
  return prompt_text

In [41]:
rag_chain_with_print = ({"context":retriever | format_docs, "question":RunnablePassthrough()}
             | prompt
             | RunnableLambda(print_prompt)
             | llm
             | StrOutputParser())

In [42]:
rag_chain_with_print.invoke("Is there doubt support ?")

Prompt -  messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Is there doubt support ? \nContext: course be taught in?Do you have a mobile application?Can I watch the class recordings or go through notes offline?Do you provide certificate for the course?Can I get invoices for reimbursement?I have more questions, how can I reach out to you?© 2025 Copyright Educosys. All rights reserved.\ncourse be taught in?Do you have a mobile application?Can I watch the class recordings or go through notes offline?Do you provide certificate for the course?Can I get invoices for reimbursement?I have more questions, how can I reach out to you?© 2025 Copyright Educosys. All rights reserved.\ninterested in AI and ML!Read moreManika KaushikSenior Software EngineerOptum-United Healt

'Yes, you can ask doubts. The context includes the question "Where can I ask you doubts?".'